In [360]:
%matplotlib inline

In [361]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy.io as sio
import scipy.sparse as sp

from matplotlib import pyplot as plt
from helpers import draw_edges, get_borderless_fig, pos_spring_layout, signed_layout, get_lcc, signed_normalized_laplacian, flatten

In [415]:
graph = 'wikiconflict'
g = nx.read_gpickle('graphs/{}.pkl'.format(graph))
A = nx.adj_matrix(g, weight='sign')
fog_data = sio.loadmat('outputs/focg-{}.mat'.format(graph))

In [416]:
df = pd.read_pickle('outputs/slashdot.pkl')